In [230]:
### pry-docの読み込み
require "/root/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-doc-0.10.0/lib/pry-doc"
#require "/root/git_jupyter_notebook/Ruby/vendor/bundle/ruby/2.3.0/gems/pry-doc-0.10.0/lib/pry-doc"
#require "/Users/ftakao2007/jupyter/vendor/bundle/ruby/2.3.0/gems/pry-doc-0.10.0/lib/pry-doc"

true

# ブロックとProc

* ブロック
    * 新たにスコープを作成する
        * for式、if式、while式などはスコープは作成されない
    * <font color="red">メソッドを呼び出すときのみ記述できる</font>
    * yieldという式を使う事でブロック内部で記述した式を呼び出すことができる
    * 「{}」と「do ～ end」の形式で書ける
        * 傾向
            * 1行で書けるときは{}
            * 複数行で書く時は do end
            
* 参考
    * [メタプログラミングRuby 第3章 まとめ　(ブロック)](http://portaltan.hatenablog.com/entry/2015/07/17/181358)

In [186]:
def func
  yield
end

puts func{ 1 }
puts func{ x = 2 }
puts func{ x = 2; y = 3}
puts func{ x = 2; y = 3; z = 4}
puts func{ x = 2; next y = 3; z = 4}
begin
  puts func{ x = 2; return y = 3; z = 4}
rescue LocalJumpError => ex
  puts ex
end

1
2
3
4
3
unexpected return


In [194]:
def func
  ary = [1,2,3]
  ary.each do |x|
    next x if x == 2
    puts x
  end
  puts "end"
end
func
puts

def func
  ary = [1,2,3]
  ary.each do |x|
    break x if x == 2
    puts x
  end
  puts "end"
end
func
puts

def func
  ary = [1,2,3]
  ary.each do |x|
    return x if x == 2
    puts x
  end
  puts "end"
end
func
puts

1
3
end

1
end

1



## ブロック内のreturn

* ブロック内でreturnすると返り値を返すのではなくブロックの処理を終えてしまう

In [71]:
def func
  ary = [1,2,3,4,5,6,7,8]
  ary.each do |a|
    return a if 0 == a % 2
    puts "1の時、ifの条件に合致しなかったためreturnはスルーされる"
    puts "a=#{a}"
    puts "2の時、条件に合致するためreturnでブロックを抜けて2を返す"
  end
end

p "func = #{func}"
puts 

def func2
  ary = [1,2,3,4,5,6,7,8]
  ary.each do |a|
    puts "条件に合致しないとき、nextで次の処理に進んでputsを実行"
    next a if 0 == a % 2
    puts a
  end
end
p "func2 = #{func2}"
puts "返り値はary"


1の時、ifの条件に合致しなかったためreturnはスルーされる
a=1
2の時、条件に合致するためreturnでブロックを抜けて2を返す
"func = 2"

条件に合致しないとき、nextで次の処理に進んでputsを実行
1
条件に合致しないとき、nextで次の処理に進んでputsを実行
条件に合致しないとき、nextで次の処理に進んでputsを実行
3
条件に合致しないとき、nextで次の処理に進んでputsを実行
条件に合致しないとき、nextで次の処理に進んでputsを実行
5
条件に合致しないとき、nextで次の処理に進んでputsを実行
条件に合致しないとき、nextで次の処理に進んでputsを実行
7
条件に合致しないとき、nextで次の処理に進んでputsを実行
"func2 = [1, 2, 3, 4, 5, 6, 7, 8]"
返り値はary


## yieldで実行結果の取得

* 「{}」に囲まれているのがブロック
    * ここのブロックは2を返す
    * メソッドfunc
        * ブロックの実行結果(yield)と引数の合計3を返している

In [29]:
def func x
  x + yield
end
puts func(1){ 2 }

### do ～ end の形式でも書ける
# func(1) do
#   2
# end

3


## スコープの生成

* ブロックの中で変数に代入された値はブロックの外では参照できない

In [167]:
def func x
  x + yield
end
puts func(1){ z = 2 }
puts

### ブロック内で作成された変数zはブロックの外では参照できない
begin
  puts z
rescue NameError => ex
  puts ex
end

3

undefined local variable or method `z' for main:Object


## クロージャとしてのブロック

* クロージャ
    * 処理の生成時の環境を束縛するもの
* おそらく
    * クロージャはブロック(メソッド)の外で作った変数を利用しているブロックのこと
    * 束縛はクロージャで使う変数をそれっぽく難しく言ったもの
* 挙動
    * ブロックの外で変数yに2を代入 (束縛yと表現する？)
    * メソッドfuncにブロックを渡し、yieldによりブロックを実行
        * このときyの値を取得して更新する
        * このyは、ブロックの外と同じy
            * <font color="red">値ではなく変数そのものが共有されている</font>
* Rubyはメソッド内部から外部の変数を参照できない
    * クロージャが呼び出し元の変数を処理に持ち込む数少ない手段の一つ

In [24]:
def func x
  x + yield
end

### ブロックの外で変数yに値を代入
y = 2

### このブロック内のyは先ほどのブロックの外の変数yと同一
puts func(1){y+=2}

### yの値は更新されている
puts y

5
4


In [195]:
def func
  yield
end

x = 3
func do
  x += 5
end

puts x

8


## ブロックの引数の指定

* funcに1と2を渡す
* func内部で以下合計する
    * 第一引数の値1
    * ブロックの実行結果
        * yieldはブロック引数2,3を合計して5を返す
* 6が返る
    * 引数を渡してブロックを処理するという順番

In [28]:
def func a, b
  a + yield(b, 3)
end

puts func(1, 2){|x,y| x+y}

6


In [196]:
def func x, y
  x + y + yield
end

func(1,2){3}

6

In [198]:
def func *x
  x.each{|i| puts i}
end
func(1,2,nil,"a",true,6)

1
2

a
true
6


[1, 2, nil, "a", true, 6]

In [202]:
def func x, *y
  y.each{|i| puts i}
  puts "x = #{x}"
end
func(1,2,nil,"a",true,6)

2

a
true
6
x = 1


## ブロックの判定(ブロックが指定されたかどうか)

* block_given?メソッドで実現できる

In [29]:
def func
  return 1 if block_given?
  2
end

puts func(){}
puts func

1
2


In [206]:
def func
  puts block_given?
end
puts func{}
puts func
puts func{nil}

true

false

true



## Procの基本

* ブロックをオブジェクトとして渡したい時に利用する
    * ブロックそのものはオブジェクトではない
* 生成
    * Procクラスのコンクラスタにブロックを指定すると生成できる
* 利用
    * Procのインスタンスに対してcallメソッドを使う


In [207]:
proc = Proc.new{|x| puts x}
proc.call(1)

1


In [96]:
proc = Proc.new{|x| puts x}
puts proc.class
puts proc.call([1,2]).class

Proc
[1, 2]
NilClass


In [100]:
proc = Proc.new{|x| puts x * 2}
puts proc.call([1,2])

[1, 2, 1, 2]



## Procオブジェクトの生成

* 遅延評価
    * 処理自体を先に生成して後で評価する
    * 例
        * カウンタで初期値をプログラム冒頭で決定 : 処理と初期値の設定
        * 後の処理でカウントする : 遅延評価

In [44]:
def get_counter start
  Proc.new do |up|
    puts "start= #{start}"
    puts "up= #{up}"
    start += up
  end
end

### 初期値として3を設定。count_upはProcオブジェクトを参照
count_up = get_counter(3)

### 初期値に1を足す
puts count_up.call(1)

### さらに3を足す
puts count_up.call(3)

start= 3
up= 1
4
start= 4
up= 3
7


In [214]:
proc1 = Proc.new{x += 1}
x = 3
puts proc1.call(x)
puts proc1.call(x)

4
5


## Proc ⇔ ブロック 変換

* Proc -> ブロック
    * Procオブジェクトに「&」をつけて最後の引数に指定する
* ブロック -> Proc
    * 最後の仮引数に「&」をつけた引数で参照できる

In [55]:
### Proc -> ブロック
def func x
  x + yield
end
proc = Proc.new{2}

### 最後の引数に&をつけて渡す
puts func(1,&proc)
  
#=========================#
  
### ブロック -> Proc
### 関数の最後の仮引数に&をつけた名前を渡す
def func2 y, &proc2
  y + proc2.call
end
puts func2(3){ 4 }

3
7


In [140]:
def func_a x, &proc1
  x + proc1.call
end
puts func_a(1){ 2 }
puts

def func_b y
  y + yield
end
puts func_b(3){ 4 }

3

7


In [217]:
def func
  yield
end

proc = Proc.new{2}
puts func(&proc)

puts func{3}

2
3


In [279]:
def func &proc1
  proc1.call
end
puts func{2}

def func &proc1
  proc(&proc1)
end
puts func{3}.call

def func
  yield
end
puts func{4}

2
3
4


In [280]:
def func &lmd1
  lambda(&lmd1)
end
puts func{1}.call

def func &lmd2
  lmd2.call
end
puts func{2}

1
2


## lambda


* lambdaメソッド
    * Procインスタンスを生成する
        * Procインスタンスはlambdaとも呼ばれる
        * Procやブロックよりもメソッドに近い動きをする
    * Proc.newで生成したインスタンはprocとも呼ばれる

In [43]:
lmd = lambda{|x| puts x}
lmd.call(1)

### Ruby1.9以降の新しい書き方 (アロー演算子)
### putsを外出し
lmd2 = -> ( x ) { x }
puts lmd2.call(2)

### putsをブロック内に記載
lmd3 = -> (x, y){puts x + y}
lmd3.call(3, 4)

1
2
7


In [115]:
proc1 = Proc.new{|x| puts x}
proc1.call(1)

lmd1 = lambda{|x| puts x}
lmd1.call(2)
  
lmd2 = -> (x){ puts x}
lmd2.call(3)

lmd3 = -> (x,y){ puts x; puts y}
lmd3.call(4,5)

1
2
3
4
5


In [257]:
proc1 = Proc.new{|x| puts x}
puts proc1.class

lmd1 = lambda{|x| puts x}
puts lmd1.class

puts Proc.ancestors

puts Proc.methods.grep(/new/)
puts Kernel.methods.grep(/proc/)
puts Kernel.methods.grep(/lambda/)

Proc
Proc
[Proc, MethodSource::MethodExtensions, MethodSource::SourceLocation::ProcExtensions, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]
[:new]
[:proc]
[:lambda]


In [231]:
show-doc Proc.new


From: proc.c (C Method):
Owner: #<Class:Proc>
Visibility: public
Signature: new(*arg1)
Number of lines: 10

Creates a new Proc object, bound to the current
context. Proc::new may be called without a block only
within a method with an attached block, in which case that block is
converted to the Proc object.

   def proc_from
     Proc.new
   end
   proc = proc_from { "hello" }
   proc.call   #=> "hello"


In [259]:
show-doc Kernel#proc


From: proc.c (C Method):
Owner: Kernel
Visibility: private
Signature: proc()
Number of lines: 1

Equivalent to Proc.new.


In [264]:
show-doc Kernel#lambda


From: proc.c (C Method):
Owner: Kernel
Visibility: private
Signature: lambda()
Number of lines: 2

Equivalent to Proc.new, except the resulting Proc objects
check the number of parameters passed when called.


## Procとlambdaの振る舞いの違い

* returnとスコープ
    * Procは生成元のスコープを脱出する
        * return以降に何らかの処理を行いたい場合はnextなどを駆使してどうにかする
    * lambdaはブロック内でreturnすると呼び出し元に復帰する

In [25]:
puts "Procはreturnでスコープを脱出するので2まで行かない"
def func_proc
  proc = Proc.new{return 1}
  puts proc.call
  puts 2
 end
puts func_proc

puts "lambdaは呼び出し元に戻るので4まで行く"
def func_lambda
  proc = lambda{return 3}
  puts proc.call
  puts 4
end
func_lambda

Procはcallでスコープを脱出するので2まで行かない
1
lambdaは呼び出し元に戻る
3
4


* 引数が一致しない場合
    * Procは無視、もしくはnilを返す
    * lambdaは例外ArgumentErrorが発生する

In [281]:
puts "引数が一致しない場合はnil"
p1 = Proc.new{|x,y| y}
p p1.call(1,2)
p p1.call(1)
p p1.call(1,2,3)
puts

puts "引数が一致しない場合は例外が発生"
l1 = lambda{|x,y| y}
p l1.call(3,4)
begin
  p l1.call(3)
rescue ArgumentError => ex
  puts ex
end

引数が一致しない場合はnil
2
nil
2

引数が一致しない場合は例外が発生
4
wrong number of arguments (given 1, expected 2)


# 配列やハッシュの走査

* for式、while式はスコープが作成されないので走査には余り使われない
    * 代わりにブロックを受け取るメソッドが使われる
* ブロックを受け取るメソッド
    * each
    * each_with_index
    * each_key
    * each_value
    * upto
    * dwonto
    * times



## 配列のeach

* each
    * メソッドのブロックの引数には要素の値が格納される
* each_with_index
    * 引数を２つ取る
        * 第一引数は要素の値
        * 第二引数はインデックス
            * 0からはじまる

In [50]:
[1,2,3].each do |value|
  puts "value = #{value}"
end

[4,5,6].each_with_index do |value, index|
  puts "index = #{index}, value = #{value}"
end

value = 1
value = 2
value = 3
index = 0, value = 4
index = 1, value = 5
index = 2, value = 6


[4, 5, 6]

In [136]:
proc1 = Proc.new{|x| puts x}
proc1.call([1,2,3])
puts
[1,2,3].each &proc1
puts


lmd1 = lambda{|v, i| puts "i = #{i}, v = #{v}"}
[10,11,12].each_with_index &lmd1
puts

[1, 2, 3]

1
2
3

i = 0, v = 10
i = 1, v = 11
i = 2, v = 12



## ハッシュのeach

* each
    * 引数を２つ取る
        * 第一引数はキー
        * 第二引数はバリュー
* each_key
    * キーのみ
* each_value
    * バリューのみ

In [61]:
{a: 1, b: 2}.each do |key, value|
  puts "key = #{key}, value = #{value}"
end

puts "引数が一つの場合はキーとバリューの配列ができる"
{a: 1, b: 2}.each do |x|
  puts "x = #{x}"
  p x[0]
end

{c: 3, d: 4}.each_key do |key|
  puts "key = #{key}"
end

{e: 5, f: 6}.each_value do |value|
  puts "value = #{value}"
end

key = a, value = 1
key = b, value = 2
引数が一つの場合はキーとバリューの配列ができる
x = [:a, 1]
:a
x = [:b, 2]
:b
key = c
key = d
value = 5
value = 6


{:e=>5, :f=>6}

In [290]:
hash = {a: "hoge", b: "fuga"}
puts hash

hash.each{|k,v| puts "k = #{k}, v = #{v}"}
hash.each{|x| puts "x = #{x}"}

{:a=>"hoge", :b=>"fuga"}
k = a, v = hoge
k = b, v = fuga
x = [:a, "hoge"]
x = [:b, "fuga"]


{:a=>"hoge", :b=>"fuga"}

## 範囲オブジェクトのeach

範囲内を繰り返す

In [139]:
("a".."d").each do |value|
  puts value
end
puts

("1"..."4").each do |value|
  puts value
end

a
b
c
d

1
2
3


"1"..."4"

In [296]:
(1..5).each{|x| puts x}
puts
("1".."5").each{|x| puts x}
puts
("a".."k").each{|x| puts x}

1
2
3
4
5

1
2
3
4
5

a
b
c
d
e
f
g
h
i
j
k


"a".."k"

## upto, downto,timesメソッド

In [71]:
2.upto(4) do |x|
  puts x
end

8.downto(6) do |x|
  puts x
end

3.times do |x|
  puts "Hello"
end

2
3
4
8
7
6
Hello
Hello
Hello


3

In [299]:
3.times{puts "hello"}

"a".upto("d"){|s| puts s}

hello
hello
hello
a
b
c
d


"a"